In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
# limit the memory ussage of GPU
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

print(tf.config.list_physical_devices('GPU'))

In [ ]:

# Dataset parameters
N_VAL = 20000 #177781
BATCH_SIZE = 1024


# load data and convert them to awkward arrays using uproot
import uproot
import awkward as ak
import tensorflow as tf
import numpy as np

file_sig = "/scratch/ucjf-atlas/njsf164/data_4top_root/user.nhidic.412043.aMcAtNloPythia8EvtGen.DAOD_PHYS.e7101_a907_r14860_p5855.4thad26_240130_v06.3_output_root.nominal.root"
file_bkg = "/scratch/ucjf-atlas/njsf164/data_4top_root/user.nhidic.410471.PhPy8EG.DAOD_PHYS.e6337_s3681_r13145_p5855.4thad26_240123_v06.2_output_root.nominal.root"

# load the tree
tree_sig = uproot.open(file_sig)['nominal']
tree_bkg = uproot.open(file_bkg)['nominal']

# load the branches
branches = ['jet_pt', 'jet_eta', 'jet_phi', 'jet_e', 'jet_tagWeightBin_DL1dv01_Continuous']
data_sig = tree_sig.arrays(branches, library='ak', entry_stop=177781)
data_bkg = tree_bkg.arrays(branches, library='ak', entry_stop=177781)


## convert awkward arrays to tensorflow tensors
tensors = []
for i,arrays in enumerate([data_sig, data_bkg]):
    tensors.append([])
    for var in branches:
        array = arrays[var]
        row_lengths = ak.num(array, axis=1).to_numpy()
        tensor = tf.RaggedTensor.from_row_lengths(ak.flatten(array, axis=None).to_numpy(), row_lengths=row_lengths, validate=False)
        tensor = tf.cast(tensor, tf.float32)
        tensors[i].append(tensor)

## calculate mean and std. dev. of the tensors
mean = [tf.reduce_mean(tf.concat([tensor_sig,tensor_bkg], axis=0)) for tensor_sig,tensor_bkg in zip(tensors[0], tensors[1])]
std_dev = [tf.math.reduce_std(tf.concat([tensor_sig,tensor_bkg], axis=0)) for tensor_sig,tensor_bkg in zip(tensors[0], tensors[1])]

for var,m,v in zip(branches,mean, std_dev):
    print(f'{var} - mean: {m.numpy()}, std_dev: {v.numpy()}')


# normalize the tensors. Subtract mean and divide by variance
for i in range(len(tensors)):
    for j in range(len(tensors[i])):
        tensors[i][j] = (tensors[i][j] - mean[j]) / std_dev[j]

## stack tensors of individual branches to create a single 3D tensor
for i in range(len(tensors)):
    tensors[i] = tf.stack(tensors[i], axis=-1)


print(tensors[0].shape)
print(tensors[1].shape)

# get the dataset size
n_total = 2*min(tensors[0].shape[0], tensors[1].shape[0])

## create signal and background datasets
sig_dataset = tf.data.Dataset.from_tensor_slices(tensors[0])
bkg_dataset = tf.data.Dataset.from_tensor_slices(tensors[1])

## add labels to the datasets
sig_dataset_labels = sig_dataset.map(lambda x: (x, 1))
bkg_dataset_labels = bkg_dataset.map(lambda x: (x, 0))

# combine datasets into a single dataset
dataset = tf.data.Dataset.sample_from_datasets( [sig_dataset_labels, bkg_dataset_labels], weights=[0.5, 0.5], stop_on_empty_dataset=True)

# split into training and validation datasets
val_dataset = dataset.take(N_VAL)
train_dataset = dataset.skip(N_VAL)

# shuffle and batch the datasets
train_dataset = train_dataset.shuffle(10000).ragged_batch(BATCH_SIZE)
val_dataset = val_dataset.ragged_batch(BATCH_SIZE)

train_dataset_size = n_total - N_VAL
print(f'Training dataset size: {train_dataset_size}')


In [ ]:
EPOCHS = 10             # number of training epochs

N_BLOCKS = 1           # number of blocks of bi-directional RNN layers
HIDDEN_SIZE = 64        # number of hidden units in the RNN layers
DROPOUT = 0.           # dropout rate

## create the input layer
inputShape = train_dataset.element_spec[0].shape  # this returns the input tensor shape including the batch dimension
print("Input shape: ", inputShape[1:])
input = tf.keras.layers.Input(shape=inputShape[1:], ragged=True) # here we provide shape without the batch dimension
layer = input

# create the mask
mask = tf.sequence_mask(row_lengths)

# we have to expand the last dimension of the input tensor to HIDDEN_SIZE
layer = tf.keras.layers.Dense(HIDDEN_SIZE)(layer)

## create the bi-directional RNN layers (using gated recurrent units, GRU)
for i in range(N_BLOCKS):
    layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(HIDDEN_SIZE, return_sequences=True, dropout=DROPOUT))(layer)
    # layer = tf.keras.layers.LayerNormalization()(layer)

## global max pooling
layer = tf.keras.layers.GlobalMaxPooling1D()(layer)

## final dense layer with sigmoid activation
layer = tf.keras.layers.Dense(1, activation='sigmoid')(layer)


## create the model
model = tf.keras.Model(inputs=input, outputs=layer)

# cosine learning rate decay
initial_learning_rate = 0.001
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate = initial_learning_rate, 
    decay_steps=EPOCHS * train_dataset_size,
    alpha=0.01 * initial_learning_rate)

# compile the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate = learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

# evaluate the model
metrices_train = model.evaluate(train_dataset)
metrices_val = model.evaluate(val_dataset)

print("")
print("Overfitting check:")
print("")
for name, value_train, value_val in zip(model.metrics_names, metrices_train, metrices_val):
    print(name, "train:", value_train, "val:", value_val)

# draw training history
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['binary_accuracy'], label='training')
plt.plot(history.history['val_binary_accuracy'], label='validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# The following function just picks data tensor from the (data, label) dataset
@ tf.function
def pick_only_data(data, label):
    return data

# draw signal and background separately
output_sig_val = model.predict(val_dataset.unbatch().filter(lambda x, label: label == 1).map(pick_only_data).ragged_batch(1024))
output_bkg_val = model.predict(val_dataset.unbatch().filter(lambda x, label: label == 0).map(pick_only_data).ragged_batch(1024))

plt.figure(figsize=(10, 4))
plt.hist(output_sig_val, histtype='step', bins=100, alpha=0.5, label='signal (val)', range=(0, 1))
plt.hist(output_bkg_val, histtype='step', bins=100, alpha=0.5, label='background (val)', range=(0, 1))

plt.xlabel('Output probability')
plt.ylabel('')
plt.legend()
plt.show()